In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("EURUSD_H1_202201030000_202504111800.csv", sep="\t")
df.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2022.01.03,00:00:00,1.13693,1.13742,1.13645,1.13726,571,0,17
1,2022.01.03,01:00:00,1.13725,1.13778,1.13687,1.13728,1345,0,7
2,2022.01.03,02:00:00,1.13728,1.13759,1.13608,1.13622,1054,0,1
3,2022.01.03,03:00:00,1.13622,1.13654,1.13497,1.13588,1349,0,1
4,2022.01.03,04:00:00,1.13588,1.13589,1.13422,1.13427,1091,0,1


In [4]:
import matplotlib.pyplot as plt

df['datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
df.set_index('datetime', inplace=True)
df = df.rename(columns={
    '<OPEN>':  'Open',
    '<HIGH>':  'High',
    '<LOW>':   'Low',
    '<CLOSE>': 'Close',
    '<TICKVOL>':'Volume',
    '<SPREAD>':'Spread'
})
df = df[['Open','High','Low','Close','Volume','Spread']]
df.head()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Rahul\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Rahul\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Rahul\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Rahul\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "c:\Users\Rahul\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [5]:
import sys

# Upgrade to the latest NumPy and pandas releases
!{sys.executable} -m pip install --upgrade numpy pandas


In [27]:
import sys

# Run this in a Jupyter cell to upgrade to the latest backtesting.py release
!{sys.executable} -m pip install --upgrade backtesting

  Using cached backtesting-0.6.4-py3-none-any.whl (191 kB)
  Attempting uninstall: backtesting
    Found existing installation: Backtesting 0.3.2
    Uninstalling Backtesting-0.3.2:
      Successfully uninstalled Backtesting-0.3.2


In [ ]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# 1) (You’ll already have loaded and prepared your H1 df with columns
#     Open, High, Low, Close, Volume, Spread and a DateTime index.)

# 2) Strategy
class EMA_RSI_ATR(Strategy):
    fast, slow           = 20, 50
    rsi_period, atr_period = 14, 14
    atr_mult, rrr        = 1.5, 2.0

    def init(self):
        # pull true pandas Series via the .s accessor
        close_s = self.data.Close.s
        high_s  = self.data.High.s
        low_s   = self.data.Low.s

        # EMA20 & EMA50
        self.ema_fast = self.I(
            lambda s: s.ewm(span=self.fast, adjust=False).mean(),
            close_s
        )
        self.ema_slow = self.I(
            lambda s: s.ewm(span=self.slow, adjust=False).mean(),
            close_s
        )

        # RSI(14)
        delta    = close_s.diff()
        gain     = delta.clip(lower=0)
        loss     = -delta.clip(upper=0)
        self.rsi = self.I(
            lambda g, l: 100 - 100/(1 + (g.ewm(alpha=1/self.rsi_period, min_periods=self.rsi_period).mean() / 
                                         l.ewm(alpha=1/self.rsi_period, min_periods=self.rsi_period).mean())),
            gain, loss
        )

        # True Range series
        tr = pd.concat([
            high_s - low_s,
            (high_s - close_s.shift()).abs(),
            (low_s  - close_s.shift()).abs()
        ], axis=1).max(axis=1)

        # ATR(14) — wrap rolling().mean() in a lambda so I() gets a Series
        self.atr = self.I(
            lambda s: s.rolling(self.atr_period).mean(),
            tr
        )

    def next(self):
        if self.position:
            return

        # Long entry
        if crossover(self.ema_fast, self.ema_slow) and self.rsi[-1] > 50:
            e = self.data.Close[-1]
            sl = e - self.atr[-1] * self.atr_mult
            tp = e + self.atr[-1] * self.atr_mult * self.rrr
            self.buy(sl=sl, tp=tp)

        # Short entry
        elif crossover(self.ema_slow, self.ema_fast) and self.rsi[-1] < 50:
            e = self.data.Close[-1]
            sl = e + self.atr[-1] * self.atr_mult
            tp = e - self.atr[-1] * self.atr_mult * self.rrr
            self.sell(sl=sl, tp=tp)

# 3) Run the backtest
bt = Backtest(
    df,
    EMA_RSI_ATR,
    cash=10_000,
    commission=0.0002,
    trade_on_close=False,
    margin=1/50
)

stats = bt.run()
bt.plot()
print(stats)


Backtest.run:   0%|          | 0/20395 [00:00<?, ?bar/s]

ImportError: cannot import name 'rewrite_warning' from 'pandas.util._exceptions' (c:\Users\Rahul\anaconda3\lib\site-packages\pandas\util\_exceptions.py)

In [24]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# 2) Define your strategy with ATR SL and TP
class EMA_RSI_ATR(Strategy):
    fast       = 20
    slow       = 50
    rsi_period = 14
    atr_period = 14
    atr_mult   = 1.5
    rrr        = 2.0

    def init(self):
        # Precompute indicators
        price = self.data.Close.s
        self.ema_fast = self.I(lambda x: x.ewm(span=self.fast, adjust=False).mean(), price)

        self.ema_slow = self.I(lambda x: x.ewm(span=self.slow, adjust=False).mean(), price)

        delta = price.diff()
        gain  = delta.clip(lower=0)
        loss  = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/self.rsi_period, min_periods=self.rsi_period).mean()
        avg_loss = loss.ewm(alpha=1/self.rsi_period, min_periods=self.rsi_period).mean()
        self.rsi = self.I(lambda g, l: 100 - (100/(1+(g/l))), avg_gain, avg_loss)

        hl = self.data.High.s - self.data.Low.s
        hc = (self.data.High - self.data.Close.s.shift()).abs()
        lc = (self.data.Low.s  - self.data.Close.s.shift()).abs()
        tr = pd.concat([hl, hc, lc], axis=1).max(axis=1)
        self.atr = self.I(pd.Series.rolling, tr, window=self.atr_period).mean()

    def next(self):
        if not self.position:
            # Bullish cross + RSI filter
            if crossover(self.ema_fast, self.ema_slow) and self.rsi[-1] > 50:
                entry = self.data.Close[-1]
                sl = entry - self.atr[-1]*self.atr_mult
                tp = entry + self.atr[-1]*self.atr_mult*self.rrr
                self.buy(sl=sl, tp=tp)
            # Bearish cross + RSI filter
            elif crossover(self.ema_slow, self.ema_fast) and self.rsi[-1] < 50:
                entry = self.data.Close[-1]
                sl = entry + self.atr[-1]*self.atr_mult
                tp = entry - self.atr[-1]*self.atr_mult*self.rrr
                self.sell(sl=sl, tp=tp)

# 3) Run the backtest
bt = Backtest(
    df,
    EMA_RSI_ATR,
    cash=10_000,
    commission=0.0002,
    trade_on_close=False,
    margin=1/50
)
stats = bt.run()
bt.plot()
print(stats)


ValueError: Indicators must return (optionally a tuple of) numpy.arrays of same length as `data` (data shape: (20410,); indicator "rolling(14)" shape: (), returned value: Rolling [window=14,center=False,axis=0,method=single])